<a href="https://colab.research.google.com/github/olanamramoj2/Personal-Projects/blob/main/Official_schedule_data_consolidator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import pandas as pd
from google.colab import drive, auth
import gspread
from google.auth import default
from googleapiclient.discovery import build
import time

# 1. Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

drive.mount('/content/drive')

# --- CONFIGURATION ---
PARENT_FOLDER_NAME = "SMPW East Zone Volunteer Availability Data"
CITY_LIST = ["Quezon City", "San Juan", "Pasig", "Mandaluyong"]
TARGET_SHEET_NAME = "Form Responses 1"
OUTPUT_COLUMNS = ["first name", "last name", "contact number", "gender", "date of duty", "preferred shift", "status", "location"]
UNIQUE_KEYS = ["first name", "last name", "date of duty", "preferred shift", "location"]
MAX_RETRIES = 3

# --- HELPER FUNCTION FOR RECURSIVE SEARCH ---
def get_all_sheets_in_folder_recursive(folder_id):
    """Finds all Google Sheets within a folder and all its subfolders."""
    all_sheets = []

    # 1. Get all Google Sheets in the current folder
    query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.spreadsheet' and trashed = false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    all_sheets.extend(results.get('files', []))

    # 2. Find all subfolders
    subfolder_query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
    subfolder_results = drive_service.files().list(q=subfolder_query, fields="files(id)").execute()
    subfolders = subfolder_results.get('files', [])

    # 3. Recursively search each subfolder
    for sf in subfolders:
        all_sheets.extend(get_all_sheets_in_folder_recursive(sf['id']))

    return all_sheets

# 3. Get Parent Folder ID
folder_query = f"name = '{PARENT_FOLDER_NAME}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
folder_results = drive_service.files().list(q=folder_query, fields="files(id)").execute()
files_found = folder_results.get('files', [])

if not files_found:
    print(f"❌ Could not find folder: {PARENT_FOLDER_NAME}")
else:
    PARENT_FOLDER_ID = files_found[0].get('id')

    # 4. Main Loop
    for city in CITY_LIST:
        print(f"\n🚀 STARTING PROCESS FOR: {city.upper()}")
        city_data = []

        # Find the City Subfolder ID
        city_folder_query = f"'{PARENT_FOLDER_ID}' in parents and name = '{city}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
        city_folder_res = drive_service.files().list(q=city_folder_query, fields="files(id)").execute()
        city_folders = city_folder_res.get('files')

        if not city_folders:
            print(f"⚠️ City folder '{city}' not found.")
            continue

        city_id = city_folders[0].get('id')

        # NEW: Get all sheets from City folder AND its subfolders
        all_city_sheets = get_all_sheets_in_folder_recursive(city_id)
        print(f"  Found {len(all_city_sheets)} total spreadsheets in {city} hierarchy.")

        for sheet_info in all_city_sheets:
            file_id = sheet_info['id']
            file_name = sheet_info['name']

            # Location extraction
            filename_without_ext = os.path.splitext(file_name)[0]
            parts = filename_without_ext.split(' - ', 1)
            location = parts[1].strip() if len(parts) > 1 else filename_without_ext
            location = location.replace(" (Responses)", "")

            attempts = 0
            success = False
            while attempts < MAX_RETRIES and not success:
                try:
                    sh = gc.open_by_key(file_id)
                    try:
                        worksheet = sh.worksheet(TARGET_SHEET_NAME)
                    except gspread.exceptions.WorksheetNotFound:
                        success = True # Skip silently if tab doesn't exist
                        continue

                    data = worksheet.get_all_records()
                    df = pd.DataFrame(data)

                    if not df.empty:
                        df.columns = df.columns.str.strip().str.lower()
                        if 'status' in df.columns:
                            status_col = df["status"].astype(str).str.strip().str.lower()
                            # open_rows = df[(status_col == "open") | (status_col == "")].copy()
                            open_rows = df[(status_col == "open")].copy()

                            if not open_rows.empty:
                                open_rows["location"] = location
                                existing_cols = [c for c in OUTPUT_COLUMNS if c in open_rows.columns]
                                city_data.append(open_rows[existing_cols])

                    success = True
                    print(f"    ✅ Read: {file_name}")

                except Exception as e:
                    attempts += 1
                    if attempts < MAX_RETRIES:
                        time.sleep(3)
                    else:
                        print(f"    ❌ [Failed] {file_name}: {e}")

        # --- SAVE CITY OUTPUT ---
        if city_data:
            new_df = pd.concat(city_data, ignore_index=True)
            new_df = new_df.drop_duplicates(subset=[k for k in UNIQUE_KEYS if k in new_df.columns], keep="last")

            clean_city_name = city.replace('/', '_')
            output_filename = f"/content/drive/MyDrive/Colab Notebooks/Schedule consolidator/{PARENT_FOLDER_NAME}/{clean_city_name}_combined_open_records.xlsx"
            new_df.to_excel(output_filename, index=False)
            print(f"💾 File Saved in: {PARENT_FOLDER_NAME}/{clean_city_name}_combined_open_records.xlsx")

print("\n✨ All cities and subfolders processed!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🚀 STARTING PROCESS FOR: QUEZON CITY
  Found 14 total spreadsheets in Quezon City hierarchy.
    ✅ Read: SMPW 2026 Availability - LRT 2 V. Mapa (Responses)
    ✅ Read: SMPW 2026 Availability - BIR Fisher Mall (Responses)
    ✅ Read: SMPW 2026 Availability - LTO Central Office (Responses)
    ✅ Read: SMPW 2026 Availability - SSS Diliman (Responses)
    ✅ Read: SMPW 2026 Availability - Quezon Memorial Circle (Responses)
    ✅ Read: SMPW 2026 Availability - LRT2 Araneta Gateway (Responses)
    ✅ Read: SMPW 2026 Availability - LRT Gilmore (Responses)
    ✅ Read: SMPW 2026 Availability - Philippine Orthopedic Center (Responses)
    ✅ Read:  SMPW 2026 Availability - NHA/ Maharlika (Responses)
    ✅ Read: SMPW 2026 Availability - LRT 2 Cubao - Diamond Arcade (Responses)
    ✅ Read: SMPW 2026 Availability - NBI Quezon City (Responses)
    ✅ Read: SMPW 2026 Availabili